In [ ]:
#!pip install -U xarray

In [1]:
import earthaccess
import rasterio
import xarray as xr

In [2]:
assert(xr.__version__ >= '2024.10.0') # for datatree support

In [3]:
earthaccess.login()

Enter your Earthdata Login username:  aimeeb
Enter your Earthdata password:  ········


In [4]:
s3fs = earthaccess.get_s3_filesystem(daac="NSIDC")

In [8]:
url = 's3://nsidc-cumulus-prod-protected/ATLAS/ATL06/006/2020/09/01/ATL06_20200901202650_10460803_006_02.h5'
ds = xr.open_datatree(s3fs.open(url), decode_times=False)
#ds = rasterio.open(s3fs.open(url, "rb"))
ds

/tmp/ipykernel_125/2933507862.py:2: UserWarning: The 'phony_dims' kwarg now defaults to 'access'. Previously 'phony_dims=None' would raise an error. For full netcdf equivalence please use phony_dims='sort'.
  ds = xr.open_datatree(s3fs.open(url), decode_times=False)


<xarray.DataTree>
Group: /
│   Attributes: (12/48)
│       short_name:                         ATL06
│       level:                              L3A
│       title:                              ATLAS/ICESat-2 L3A Land Ice Height
│       description:                        This data set (ATL06) provides geoloc...
│       Conventions:                        CF-1.6
│       contributor_name:                   Thomas A Neumann (thomas.neumann@nasa...
│       ...                                 ...
│       processing_level:                   3A
│       references:                         http://nsidc.org/data/icesat2/data.html
│       project:                            ICESat-2 > Ice, Cloud, and land Eleva...
│       instrument:                         ATLAS > Advanced Topographic Laser Al...
│       platform:                           ICESat-2 > Ice, Cloud, and land Eleva...
│       source:                             Spacecraft
├── Group: /ancillary_data
│   │   Dimensions:              (phony_dim_0: 1)
│   │   Dimensions without coordinates: phony_dim_0
│   │   Data variables: (12/25)
│   │       atlas_sdp_gps_epoch  (phony_dim_0) float64 8B ...
│   │       control              (phony_dim_0) |S100000 100kB ...
│   │       data_end_utc         (phony_dim_0) |S27 27B ...
│   │       data_start_utc       (phony_dim_0) |S27 27B ...
│   │       end_cycle            (phony_dim_0) int32 4B ...
│   │       end_delta_time       (phony_dim_0) float64 8B ...
│   │       ...                   ...
│   │       start_gpssow         (phony_dim_0) float64 8B ...
│   │       start_gpsweek        (phony_dim_0) int32 4B ...
│   │       start_orbit          (phony_dim_0) int32 4B ...
│   │       start_region         (phony_dim_0) int32 4B ...
│   │       start_rgt            (phony_dim_0) int32 4B ...
│   │       version              (phony_dim_0) |S80 80B ...
│   │   Attributes:
│   │       Description:  Contains information ancillary to the data product. This ma...
│   │       data_rate:    Data within this group pertain to the granule in its entirety.
│   └── Group: /ancillary_data/land_ice
│           Dimensions:          (phony_dim_1: 1, phony_dim_2: 6)
│           Dimensions without coordinates: phony_dim_1, phony_dim_2
│           Data variables: (12/24)
│               dt_hist          (phony_dim_1) float64 8B ...
│               fit_maxiter      (phony_dim_1) int32 4B ...
│               fpb_maxiter      (phony_dim_1) int32 4B ...
│               max_res_ids      (phony_dim_1) int32 4B ...
│               min_dist         (phony_dim_1) float32 4B ...
│               min_gain_th      (phony_dim_1) float32 4B ...
│               ...               ...
│               qs_lim_sss       (phony_dim_1) int32 4B ...
│               rbin_width       (phony_dim_1) float32 4B ...
│               sigma_beam       (phony_dim_1) float32 4B ...
│               sigma_tx         (phony_dim_2) float32 24B ...
│               t_dead           (phony_dim_2) float32 24B ...
│               txp_maxiter      (phony_dim_1) int32 4B ...
│           Attributes:
│               Description:  Contains land-ice-specific  information ancillary to the da...
├── Group: /orbit_info
│       Dimensions:                (phony_dim_3: 47, crossing_time: 1, sc_orient_time: 1)
│       Coordinates:
│         * crossing_time          (crossing_time) float64 8B 8.423e+07
│         * sc_orient_time         (sc_orient_time) float64 8B 8.415e+07
│       Dimensions without coordinates: phony_dim_3
│       Data variables:
│           bounding_polygon_lat1  (phony_dim_3) float64 376B ...
│           bounding_polygon_lon1  (phony_dim_3) float64 376B ...
│           cycle_number           (crossing_time) int8 1B ...
│           lan                    (crossing_time) float64 8B ...
│           orbit_number           (crossing_time) uint16 2B ...
│           rgt                    (crossing_time) int16 2B ...
│           sc_orient              (sc_orient_time) int8 1B ...
│       Attributes:
│ 

In [6]:
ds.data_vars.keys()

KeysView(Data variables:
    *empty*)

In [39]:
import rasterio
from rasterio.warp import transform_bounds
import morecantile
import math

with rasterio.open(s3fs.open(url, "rb")) as src:
    tms = morecantile.tms.get("WebMercatorQuad")
    
    # Get native resolution
    pixel_size = src.res[0]
    
    # Transform bounds to WGS84
    bounds_4326 = transform_bounds(
        src.crs, 'EPSG:4326',
        src.bounds.left, src.bounds.bottom,
        src.bounds.right, src.bounds.top
    )
    
    center_lon = (bounds_4326[0] + bounds_4326[2]) / 2
    center_lat = (bounds_4326[1] + bounds_4326[3]) / 2
    
    # Find best zoom
    best_zoom = None
    best_diff = float('inf')
    
    for zoom in range(8, 19):
        tile_res = (156543.03 * math.cos(math.radians(center_lat))) / (2 ** zoom)
        diff = abs(tile_res - pixel_size)
        if diff < best_diff:
            best_diff = diff
            best_zoom = zoom
    
    # Get center tile
    tile = tms.tile(center_lon, center_lat, best_zoom)
    
    print(f"Resolution: {pixel_size:.2f} m/pixel")
    print(f"Best zoom: {best_zoom}")
    print(f"Sample tile: z={tile.z}, x={tile.x}, y={tile.y}")
    print(f"URL: /{tile.z}/{tile.x}/{tile.y}")

Resolution: 30.00 m/pixel
Best zoom: 12
Sample tile: z=12, x=3934, y=2709
URL: /12/3934/2709
